In [3]:
import random
from IPython.display import clear_output


# Simple Genetic Algorithm



# Example Equation
def foo(x,y,z):
    return 6 * x**3 + 9 * y**2 + 10 * z - 25
 

def fitness(x, y, z):
    ans = foo(x,y,z)
    if ans == 0:
        return 999999
    else:
        return abs(1/ans)

    
search_range = [0,10000]
Gen_diff = 0.02
    
# Generate Solutions
solutions = []
for s in range(1000):
    solutions.append((random.uniform(search_range[0],search_range[1]),
                      random.uniform(search_range[0],search_range[1]),
                      random.uniform(search_range[0],search_range[1]),
                    ))
                    
results_show = 20
for i in range(10000):
    
    
    ranked_solutions = []
    for s in solutions:
        ranked_solutions.append((fitness(s[0],s[1],s[2]),s))
        ranked_solutions.sort()
        ranked_solutions.reverse()
    print(f"@@@@@@@@@@ Gen {i} best solutions @@@@@@@@@@@")
    print(ranked_solutions[0])
    
    if ranked_solutions[0][0] > 9999:
        break
    if i % results_show == 0:
        clear_output(wait=True)
    
    best_solutions = ranked_solutions[:100]
    elements_x = []
    elements_y = []
    elements_z = []
    for s in best_solutions:
        elements_x.append(s[1][0])
        elements_y.append(s[1][1])
        elements_z.append(s[1][2])
        
    new_Gen = []
    for _ in range(1000):
        e1 = random.choice(elements_x) * random.uniform(1-Gen_diff,1+Gen_diff)
        e2 = random.choice(elements_y) * random.uniform(1-Gen_diff,1+Gen_diff)
        e3 = random.choice(elements_z) * random.uniform(1-Gen_diff,1+Gen_diff)
        new_Gen.append((e1,e2,e3))
    solutions = new_Gen
        

@@@@@@@@@@ Gen 321 best solutions @@@@@@@@@@@
(0.13616550844419165, (0.34688363632771074, 0.3763384804155696, 3.081888842902548))
@@@@@@@@@@ Gen 322 best solutions @@@@@@@@@@@
(0.1502545386200906, (0.3179083874745905, 0.37493290967896453, 3.019742293876776))
@@@@@@@@@@ Gen 323 best solutions @@@@@@@@@@@
(0.1625897202769224, (0.3136014494592987, 0.3554843332923987, 2.982807989425874))
@@@@@@@@@@ Gen 324 best solutions @@@@@@@@@@@
(0.18080139141056045, (0.30896982650383337, 0.36235077942716176, 2.9172278178779667))
@@@@@@@@@@ Gen 325 best solutions @@@@@@@@@@@
(0.20280219281173803, (0.32508968797340204, 0.3598509267140347, 2.8559339631602976))
@@@@@@@@@@ Gen 326 best solutions @@@@@@@@@@@
(0.23185466389658002, (0.3393044926529287, 0.34657600278618067, 2.7997632652107423))
@@@@@@@@@@ Gen 327 best solutions @@@@@@@@@@@
(0.2603123074916173, (0.30552419873997655, 0.35406727613208194, 2.7542151727313207))
@@@@@@@@@@ Gen 328 best solutions @@@@@@@@@@@
(0.2968002155869893, (0.31457378918549433,

In [3]:
import pygame
import random
from enum import Enum
from collections import namedtuple

pygame.init()
font = pygame.font.Font('arial.ttf', 25)
#font = pygame.font.SysFont('arial', 25)

class Direction(Enum):
    RIGHT = 1
    LEFT = 2
    UP = 3
    DOWN = 4
    
Point = namedtuple('Point', 'x, y')

# rgb colors
WHITE = (255, 255, 255)
RED = (200,0,0)
BLUE1 = (0, 0, 255)
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPEED = 20

class SnakeGame:
    
    def __init__(self, w=640, h=480):
        self.w = w
        self.h = h
        # init display
        self.display = pygame.display.set_mode((self.w, self.h))
        pygame.display.set_caption('Snake')
        self.clock = pygame.time.Clock()
        
        # init game state
        self.direction = Direction.RIGHT
        
        self.head = Point(self.w/2, self.h/2)
        self.snake = [self.head, 
                      Point(self.head.x-BLOCK_SIZE, self.head.y),
                      Point(self.head.x-(2*BLOCK_SIZE), self.head.y)]
        
        self.score = 0
        self.food = None
        self._place_food()
        
    def _place_food(self):
        x = random.randint(0, (self.w-BLOCK_SIZE )//BLOCK_SIZE )*BLOCK_SIZE 
        y = random.randint(0, (self.h-BLOCK_SIZE )//BLOCK_SIZE )*BLOCK_SIZE
        self.food = Point(x, y)
        if self.food in self.snake:
            self._place_food()
        
    def play_step(self):
        # 1. collect user input
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    self.direction = Direction.LEFT
                elif event.key == pygame.K_RIGHT:
                    self.direction = Direction.RIGHT
                elif event.key == pygame.K_UP:
                    self.direction = Direction.UP
                elif event.key == pygame.K_DOWN:
                    self.direction = Direction.DOWN
        
        # 2. move
        self._move(self.direction) # update the head
        self.snake.insert(0, self.head)
        
        # 3. check if game over
        game_over = False
        if self._is_collision():
            game_over = True
            return game_over, self.score
            
        # 4. place new food or just move
        if self.head == self.food:
            self.score += 1
            self._place_food()
        else:
            self.snake.pop()
        
        # 5. update ui and clock
        self._update_ui()
        self.clock.tick(SPEED)
        # 6. return game over and score
        return game_over, self.score
    
    def _is_collision(self):
        # hits boundary
        if self.head.x > self.w - BLOCK_SIZE or self.head.x < 0 or self.head.y > self.h - BLOCK_SIZE or self.head.y < 0:
            return True
        # hits itself
        if self.head in self.snake[1:]:
            return True
        
        return False
        
    def _update_ui(self):
        self.display.fill(BLACK)
        
        for pt in self.snake:
            pygame.draw.rect(self.display, BLUE1, pygame.Rect(pt.x, pt.y, BLOCK_SIZE, BLOCK_SIZE))
            pygame.draw.rect(self.display, BLUE2, pygame.Rect(pt.x+4, pt.y+4, 12, 12))
            
        pygame.draw.rect(self.display, RED, pygame.Rect(self.food.x, self.food.y, BLOCK_SIZE, BLOCK_SIZE))
        
        text = font.render("Score: " + str(self.score), True, WHITE)
        self.display.blit(text, [0, 0])
        pygame.display.flip()
        
    def _move(self, direction):
        x = self.head.x
        y = self.head.y
        if direction == Direction.RIGHT:
            x += BLOCK_SIZE
        elif direction == Direction.LEFT:
            x -= BLOCK_SIZE
        elif direction == Direction.DOWN:
            y += BLOCK_SIZE
        elif direction == Direction.UP:
            y -= BLOCK_SIZE
            
        self.head = Point(x, y)
            

if __name__ == '__main__':
    game = SnakeGame()
    
    # game loop
    while True:
        game_over, score = game.play_step()
        
        if game_over == True:
            break
        
    print('Final Score', score)
        
        
    pygame.quit()

Final Score 0


In [6]:
from IPython.display import clear_output
print('124151412412')
clear_output(wait=True)
print('key')

key
